# 🧪 에이전트 통합 데모 (rules_core 연동, ipynb)

**목표**: 기존 **룰베이스 추천기(`rules_core.py`)** 위에 **에이전트 레이어**를 얹어,
의사(시뮬레이션) 피드백 → 랭커 학습 → 추론(룰 근방 리랭킹)을 한 번에 확인합니다.

**사전 준비**
- 프로젝트 루트에 아래 파일이 있어야 합니다.
  - `rules_core.py`  ← 여러분의 기존 룰 코드(설문→점수→배합)
  - `agent_core.py`  ← 에이전트 핵심(보상/피처/데이터셋/랭커)
  - `agent_adapter.py` ← 룰↔에이전트 연결(설문→Context, 룰 훅)
- 파이썬 패키지: `scikit-learn` (없다면 아래 설치 셀 참고)

In [17]:
# (선택) 설치 — 환경에 따라 필요할 수 있습니다.
%pip install scikit-learn ipywidgets

Note: you may need to restart the kernel to use updated packages.


## 1) 모듈 임포트 & 환경 점검

In [18]:
# 🔧 모듈 로드
from importlib import import_module

try:
    rules_core = import_module('rules_core')
except Exception as e:
    raise RuntimeError("rules_core.py(여러분의 룰 코드)를 프로젝트 루트에 두고 다시 실행하세요.") from e

try:
    agent_core   = import_module('agent_core')
    agent_adapter = import_module('agent_adapter')
except Exception as e:
    raise RuntimeError("agent_core.py / agent_adapter.py 가 누락되었습니다. 프로젝트 루트에 저장해 주세요.") from e

assert hasattr(rules_core, 'GRAINS')
assert hasattr(rules_core, 'WEIGHTS')
assert hasattr(rules_core, 'generate_candidates')

print("✔ rules_core / agent_core / agent_adapter import OK")
print("GRAINS shape:", getattr(rules_core.GRAINS, 'shape', None))
print("WEIGHTS keys (sample):", list(rules_core.WEIGHTS.keys())[:8])

# 카탈로그 미리보기(선택)
rules_core.GRAINS.head()


✔ rules_core / agent_core / agent_adapter import OK
GRAINS shape: (16, 9)
WEIGHTS keys (sample): ['base_rules', 'purpose', 'texture_taste', 'constitution_gut', 'frequency', 'allergen', 'caps', 'mins']


,곡물,태그,섬유,단백,부드러움,쫀득,고소,GI,글루텐
0,백미,"[base, soft, mild]",0,0,3,0,0,3,0
1,현미,"[base, chewy, nutty]",2,1,1,2,1,2,0
2,찹쌀,"[base, soft, sticky]",0,0,3,0,0,3,0
3,보리,"[base, fiber, chewy]",3,1,1,2,1,1,1
4,귀리,"[fiber, nutty]",3,2,1,1,2,2,0


## 2) 설문 값공간 전수 + 폭 제한

In [19]:
import itertools

def _powerset_limited(opts, k_max):
    out = [tuple()]  # 공집합
    for k in range(1, k_max+1):
        out.extend(itertools.combinations(opts, k))
    return out

def build_value_space_from_schema_exhaustive(schema: dict, grains_df, multiselect_mode='powerset_k', k_max=2):
    """
    스키마 기반 전수용 값공간 생성.
    - dropdown: options 전부
    - int: min..max..step
    - checkbox: [False, True]
    - multiselect(grains): '없음' + (곡물에서 0~k_max개 조합)  ← 폭 제한
    반환: dict[name] = list-of-values (멀티셀렉트는 list로)
    """
    grains = sorted(grains_df["곡물"].astype(str).tolist())
    space = {}
    for sec in schema.get("sections", []):
        t, name = sec.get("type"), sec.get("name")
        if not t or not name:
            continue

        if t == "dropdown":
            space[name] = list(sec.get("options", []))

        elif t == "int":
            vmin, vmax, step = int(sec.get("min", 0)), int(sec.get("max", 10)), int(sec.get("step", 1))
            space[name] = list(range(vmin, vmax+1, step))

        elif t == "checkbox":
            space[name] = [False, True]

        elif t == "multiselect":
            if sec.get("options_from") == "grains":
                base = [g for g in grains]  # 곡물 전체
            else:
                base = list(sec.get("options", []))
            # 전수 폭을 줄이기 위해: '없음' + 0~k개 조합
            if multiselect_mode == 'powerset_k':
                tuples = _powerset_limited(base, k_max)  # () , (a), (a,b) ...
                lists  = [list(t) for t in tuples]
                space[name] = [["없음"]] + [lst for lst in lists if lst]  # 공집합 대신 '없음' 사용
            else:
                # 단일선택만(폭 더 줄이기)
                space[name] = [["없음"]] + [[g] for g in base]

    # 하위 호환 필드(스키마에 없으면 기본 후보 1개)
    for k, v in [("체질", ["보통"]), ("장건강", ["보통"]), ("맛", ["담백/중성"])]:
        space.setdefault(k, v)

    return space

def estimate_combinations(space: dict) -> int:
    n = 1
    for v in space.values():
        n *= max(1, len(v))
    return n

def iter_all_surveys(space: dict):
    """
    전수 조합을 하나씩 yield (메모리 안전)
    멀티셀렉트 값은 이미 list 형태여야 함.
    """
    keys = list(space.keys())
    vals = [space[k] for k in keys]
    for tpl in itertools.product(*vals):
        s = dict(zip(keys, tpl))
        # 안전 보강(규칙에서 쓰는 기본값)
        s.setdefault("체질", "보통")
        s.setdefault("장건강", "보통")
        s.setdefault("맛", "담백/중성")
        # '없음' → 실제 무선택
        if isinstance(s.get("기피곡물"), list) and s["기피곡물"] == ["없음"]:
            s["기피곡물"] = ["없음"]  # 여러분 규칙이 "없음" 문자열을 기대하므로 그대로 둠
        yield s



In [20]:
# === 값공간 유틸 ===
import itertools

def _powerset_limited(opts, k_max):
    out = [tuple()]  # 공집합
    for k in range(1, k_max+1):
        out.extend(itertools.combinations(opts, k))
    return out

def build_value_space_from_schema_exhaustive(schema: dict, grains_df, multiselect_mode='powerset_k', k_max=2):
    grains = sorted(grains_df["곡물"].astype(str).tolist())
    space = {}
    for sec in schema.get("sections", []):
        t, name = sec.get("type"), sec.get("name")
        if not t or not name:
            continue

        if t == "dropdown":
            space[name] = list(sec.get("options", []))
        elif t == "int":
            vmin, vmax, step = int(sec.get("min", 0)), int(sec.get("max", 10)), int(sec.get("step", 1))
            space[name] = list(range(vmin, vmax+1, step))
        elif t == "checkbox":
            space[name] = [False, True]
        elif t == "multiselect":
            if sec.get("options_from") == "grains":
                base = [g for g in grains]
            else:
                base = list(sec.get("options", []))
            if multiselect_mode == 'powerset_k':
                tuples = _powerset_limited(base, k_max)
                lists  = [list(t) for t in tuples]
                space[name] = [["없음"]] + [lst for lst in lists if lst]  # 공집합 대신 '없음'
            else:
                space[name] = [["없음"]] + [[g] for g in base]

    for k, v in [("체질", ["보통"]), ("장건강", ["보통"]), ("맛", ["담백/중성"])]:
        space.setdefault(k, v)
    return space

def estimate_combinations(space: dict) -> int:
    n = 1
    for v in space.values():
        n *= max(1, len(v))
    return n

def iter_all_surveys(space: dict):
    keys = list(space.keys())
    vals = [space[k] for k in keys]
    for tpl in itertools.product(*vals):
        s = dict(zip(keys, tpl))
        s.setdefault("체질", "보통")
        s.setdefault("장건강", "보통")
        s.setdefault("맛", "담백/중성")
        if isinstance(s.get("기피곡물"), list) and s["기피곡물"] == ["없음"]:
            s["기피곡물"] = ["없음"]
        yield s

# === 값공간 구성 & 전수 ===
space = build_value_space_from_schema_exhaustive(
    rules_core.SURVEY_SCHEMA,
    rules_core.GRAINS,
    multiselect_mode='powerset_k',
    k_max=2
)

# 전수 폭 줄이기: 곡물 수 고정
space["곡물 수"] = [5]

print("예상 전수 개수:", estimate_combinations(space))

surveys_all = []
for i, s in enumerate(iter_all_surveys(space), 1):
    surveys_all.append(s)
    if i % 10000 == 0:
        print("진행:", i)
print("총 전수:", len(surveys_all))
print(surveys_all[0])


예상 전수 개수: 1420416
진행: 10000
진행: 20000
진행: 30000
진행: 40000
진행: 50000
진행: 60000
진행: 70000
진행: 80000
진행: 90000
진행: 100000
진행: 110000
진행: 120000
진행: 130000
진행: 140000
진행: 150000
진행: 160000
진행: 170000
진행: 180000
진행: 190000
진행: 200000
진행: 210000
진행: 220000
진행: 230000
진행: 240000
진행: 250000
진행: 260000
진행: 270000
진행: 280000
진행: 290000
진행: 300000
진행: 310000
진행: 320000
진행: 330000
진행: 340000
진행: 350000
진행: 360000
진행: 370000
진행: 380000
진행: 390000
진행: 400000
진행: 410000
진행: 420000
진행: 430000
진행: 440000
진행: 450000
진행: 460000
진행: 470000
진행: 480000
진행: 490000
진행: 500000
진행: 510000
진행: 520000
진행: 530000
진행: 540000
진행: 550000
진행: 560000
진행: 570000
진행: 580000
진행: 590000
진행: 600000
진행: 610000
진행: 620000
진행: 630000
진행: 640000
진행: 650000
진행: 660000
진행: 670000
진행: 680000
진행: 690000
진행: 700000
진행: 710000
진행: 720000
진행: 730000
진행: 740000
진행: 750000
진행: 760000
진행: 770000
진행: 780000
진행: 790000
진행: 800000
진행: 810000
진행: 820000
진행: 830000
진행: 840000
진행: 850000
진행: 860000
진행: 870000
진행: 880000
진행: 890000
진행: 900000
진

## 3) 층화 샘플링 (학습 표본 균형)

In [21]:
from collections import defaultdict, Counter
import random

def stratified_sample_from_surveys(surveys, key: str, per_class: int = 800, seed: int = 42):
    random.seed(seed)
    buckets = defaultdict(list)
    for s in surveys:
        cls = s.get(key)
        buckets[cls].append(s)
        if len(buckets[cls]) > per_class * 3:
            buckets[cls] = random.sample(buckets[cls], per_class * 2)

    out = []
    for cls, items in buckets.items():
        if len(items) <= per_class:
            out.extend(items)
        else:
            out.extend(random.sample(items, per_class))
    random.shuffle(out)
    return out

# 👇 필요에 따라 per_class 조절 (과하면 시간이 오래 걸립니다)
subset = stratified_sample_from_surveys(surveys_all, key="취식 목적", per_class=10000, seed=7)
print("subset 크기:", len(subset))
print("분포:", Counter([s["취식 목적"] for s in subset]))


subset 크기: 60000
분포: Counter({'체중 관리': 10000, '맛중심': 10000, '심혈관 건강': 10000, '혈당관리': 10000, '장 건강': 10000, '근육 및 에너지 보강': 10000})


## 4) ⚠️ 보상함수 패치 (배합 의존형 + reward 반환)

In [22]:
# === 믹스 의존형 compute_metrics (노트북에서 임시 패치) ===
import numpy as np
import pandas as pd

FEATURE_MAP = {
    "gi": "GI",          # 예: GI (낮을수록 좋음)
    "fiber": "섬유",      # 예: 섬유
    "protein": "단백",    # 예: 단백질
    "soft": "부드러움",
    "sticky": "쫀득",
    "nutty": "고소",
    "gluten": "글루텐",   # 1/0 또는 True/False
}

def _col(df, key, default=0.0):
    col = FEATURE_MAP.get(key, key)
    return df[col].astype(float).values if col in df.columns else np.full(len(df), default, float)

def _mix_vector(mix, grains_df):
    index = grains_df["곡물"].astype(str).tolist()
    vec = np.zeros(len(index), float)
    for g, pct in (mix or {}).items():
        try:
            i = index.index(str(g))
            vec[i] = float(pct)
        except ValueError:
            pass
    if vec.sum() <= 0:
        return np.zeros_like(vec)
    return vec / vec.sum()

# --- Context 안전 접근 유틸 ---
def ctx_pick(ctx, keys, default=None):
    """
    ctx가 dict일 수도, 객체일 수도 있으므로
    keys(우선순위 리스트)를 순서대로 탐색해서 첫 매칭 값을 반환.
    """
    if ctx is None:
        return default
    for k in keys:
        # dict
        if isinstance(ctx, dict) and k in ctx:
            return ctx[k]
        # 객체 속성
        if hasattr(ctx, k):
            return getattr(ctx, k)
    return default

# --- 믹스 의존형 보상 함수 (ctx 객체 대응으로 수정) ---
def compute_metrics_mix_dependent(grains_df, mix, ctx=None):
    w = _mix_vector(mix, grains_df)

    gi      = np.dot(w, _col(grains_df, "gi", default=55))
    fiber   = np.dot(w, _col(grains_df, "fiber", default=0.0))
    protein = np.dot(w, _col(grains_df, "protein", default=0.0))
    soft    = np.dot(w, _col(grains_df, "soft", default=0.0))
    sticky  = np.dot(w, _col(grains_df, "sticky", default=0.0))
    nutty   = np.dot(w, _col(grains_df, "nutty", default=0.0))
    gluten  = np.dot(w, _col(grains_df, "gluten", default=0.0))

    # 목적 가중치
    wp = {
        "혈당관리":           dict(gi=1.2, fiber=1.0, protein=0.4),
        "체중 관리":          dict(gi=0.9, fiber=1.1, protein=0.6),
        "근육 및 에너지 보강": dict(gi=0.4, fiber=0.6, protein=1.2),
        "심혈관 건강":        dict(gi=0.6, fiber=1.1, protein=0.6),
        "장 건강":            dict(gi=0.4, fiber=1.2, protein=0.4),
        "맛중심":             dict(gi=0.5, fiber=0.5, protein=0.5),
    }
    # ✅ dict/객체 모두 지원
    obj = ctx_pick(ctx, ["purpose", "취식 목적", "목적"], default="맛중심")
    pw = wp.get(obj, wp["맛중심"])

    # 식감 선호 반영 (dict/객체 모두 지원)
    taste_pref = ctx_pick(ctx, ["taste_pref", "선호식감/맛", "식감"], default=None)
    if taste_pref == "고슬밥":
        taste_term = 0.6*soft + 0.2*sticky
    elif taste_pref == "찰진밥":
        taste_term = 0.6*sticky + 0.2*soft
    else:
        taste_term = 0.4*soft + 0.4*sticky

    reward = (
        - pw["gi"]      * gi/100.0 +
          pw["fiber"]   * fiber/10.0 +
          pw["protein"] * protein/10.0 +
        + 0.3           * taste_term +
        - 0.5           * gluten
    )

    return {
        "gi": float(gi), "fiber": float(fiber), "protein": float(protein),
        "soft": float(soft), "sticky": float(sticky), "nutty": float(nutty),
        "gluten": float(gluten),
        "reward": float(reward),
    }


# ⚠️ 이 셀 이후에는 compute_metrics(...)가 “배합 의존형 + reward 포함”으로 동작하도록 래핑
def compute_metrics(grains_df, mix, ctx=None):
    return compute_metrics_mix_dependent(grains_df, mix, ctx)

# === 통일 패치(몽키 패치): 학습과 평가가 같은 보상식/지표를 쓰도록 강제
import agent_core, agent_adapter

def unified_compute_metrics(grains_df, mix, ctx=None):
    return compute_metrics_mix_dependent(grains_df, mix, ctx)

def unified_reward(grains_df, mix, ctx):
    return unified_compute_metrics(grains_df, mix, ctx)["reward"]

agent_core.compute_metrics = unified_compute_metrics
agent_core.reward = unified_reward
agent_adapter.compute_metrics = unified_compute_metrics  # (안전용)

# 선택: 제대로 패치됐는지 확인
print("agent_core.compute_metrics ->", agent_core.compute_metrics.__name__)
print("agent_core.reward ->", agent_core.reward.__name__)

agent_core.compute_metrics -> unified_compute_metrics
agent_core.reward -> unified_reward


## 5) Pairwise 데이터 생성 → 랭커 학습/평가

In [23]:
from agent_adapter import make_pairwise_dataset_from_surveys
import numpy as np

N_PAIRS    = 8    # 설문당 pair 수 (데이터량/시간 조절)
RAND_SCALE = 8.0  # 후보 다양성

grains_df, X, y = make_pairwise_dataset_from_surveys(
    subset,
    n_pairs_per_survey=N_PAIRS,
    rand_scale=RAND_SCALE,
    seed=42
)
X = X.astype(float); y = y.astype(int)
print("X shape:", X.shape, "| y:", y.shape)

from agent_core import Ranker
ranker = Ranker().fit(X, y)
print("AUC(train):", ranker.score(X, y))


X shape: (960000, 14) | y: (960000,)
AUC(train): 0.5913596263802083


## 6) 추론 데모 (룰 R₀ vs 에이전트)

In [24]:
from agent_adapter import map_survey_to_context, rule_recommend_from_rules_core
from agent_core import project_constraints
import random

test_survey = random.choice(subset)  # ✅ 학습 분포와 일치
ctx = map_survey_to_context(test_survey)

r0 = rule_recommend_from_rules_core(ctx, grains_df)
best = ranker.infer_best(grains_df, ctx, rule_recommend_from_rules_core,
                         n_candidates=12, rand_scale=6.0)
best = project_constraints(best, grains_df, ctx)

print("[테스트 설문]", test_survey)
print("\n[룰 배합 R0]", r0)
print("\n[에이전트 추천 배합]", best)

m0 = compute_metrics(grains_df, r0, ctx)     # ✅ ctx 전달
m1 = compute_metrics(grains_df, best, ctx)   # ✅ ctx 전달
print("\n[지표 R0]", m0)
print("[지표 Best]", m1)
print("Δ reward:", round(m1["reward"] - m0["reward"], 4))


[테스트 설문] {'취식 목적': '장 건강', '현재 건강 이슈': '임신/수유', '나이': '50대 이상', '성별': '여', '선호식감/맛': '고슬밥', '섭취 빈도': '매일', '곡물 수': 5, '기피곡물': ['율무', '퀴노아'], '알레르겐 회피(글루텐)': False, '체질': '보통', '장건강': '보통', '맛': '담백/중성'}

[룰 배합 R0] {'렌틸콩': np.float64(15.0), '귀리': np.float64(15.0), '기장': np.float64(15.0), '서리태': np.float64(15.0), '백미': np.float64(40.0)}

[에이전트 추천 배합] {'렌틸콩': np.float64(15.0), '귀리': np.float64(15.0), '기장': np.float64(18.85375485573114), '서리태': np.float64(15.0), '백미': np.float64(36.14624514426886)}

[지표 R0] {'gi': 2.1, 'fiber': 1.3499999999999999, 'protein': 1.3499999999999999, 'soft': 1.95, 'sticky': 0.6, 'nutty': 0.9, 'gluten': 0.0, 'reward': 0.4305}
[지표 Best] {'gi': 2.0614624514426887, 'fiber': 1.3885375485573113, 'protein': 1.3885375485573113, 'soft': 1.9114624514426886, 'sticky': 0.6385375485573114, 'nutty': 0.9385375485573114, 'gluten': 0.0, 'reward': 0.43454644259851766}
Δ reward: 0.004


## 7) 후보 다양성/투영 점검 (선택)

In [25]:
from copy import deepcopy

def probe_candidates(grains_df, ctx, n_candidates=12, rand_scale=6.0):
    base = rule_recommend_from_rules_core(ctx, grains_df)
    cands = [base]
    for _ in range(n_candidates-1):
        cand = deepcopy(base)
        import random
        keys = list(cand.keys())
        a, b = random.sample(keys, 2)
        delta = random.uniform(-rand_scale, rand_scale)
        cand[a] = max(0, cand[a] + delta)
        cand[b] = max(0, cand[b] - delta)
        s = sum(cand.values())
        if s > 0:
            for k in cand: cand[k] = cand[k] * 100.0 / s
        cands.append(cand)
    return cands

cands = probe_candidates(grains_df, ctx, n_candidates=12, rand_scale=6.0)
print("unique candidates:", len({tuple(sorted(d.items())) for d in cands}))

proj = [project_constraints(c, grains_df, ctx) for c in cands]
print("unique after project:", len({tuple(sorted(d.items())) for d in proj}))


unique candidates: 12
unique after project: 12


## 8) A/B 리포트 (안전 버전)

In [26]:
import random
from statistics import mean

def ab_report(grains_df, ranker, surveys, n=100, n_candidates=12, rand_scale=6.0):
    wins = 0
    r0_rewards, best_rewards = [], []
    for _ in range(n):
        s = random.choice(surveys)
        ctx = map_survey_to_context(s)
        r0 = rule_recommend_from_rules_core(ctx, grains_df)
        best = ranker.infer_best(grains_df, ctx, rule_recommend_from_rules_core,
                                 n_candidates=n_candidates, rand_scale=rand_scale)
        best = project_constraints(best, grains_df, ctx)

        r0_reward   = compute_metrics(grains_df, r0, ctx)["reward"]     # ✅ ctx 전달
        best_reward = compute_metrics(grains_df, best, ctx)["reward"]   # ✅ ctx 전달

        r0_rewards.append(r0_reward)
        best_rewards.append(best_reward)
        wins += (best_reward > r0_reward)

    print(f"[A/B] N={n}")
    print(f"- R0 mean reward:   {mean(r0_rewards):.4f}")
    print(f"- Best mean reward: {mean(best_rewards):.4f}")
    print(f"- Win rate (Best > R0): {wins/n:.1%}")

ab_report(grains_df, ranker, subset, n=100, n_candidates=12, rand_scale=6.0)


[A/B] N=100
- R0 mean reward:   0.4034
- Best mean reward: 0.4130
- Win rate (Best > R0): 100.0%
